In [5]:
from wiki_dump_reader import Cleaner, iterate

corpus = []
cleaner = Cleaner()
for title, text in iterate('data/enwiki-latest-pages-articles.xml'):
    text = cleaner.clean_text(text)
    cleaned_text, links = cleaner.build_links(text)
    corpus.append(cleaned_text)


In [6]:
long_text = []
for i in corpus:
    if len(i) > 250:
        long_text.append(i)
    

In [7]:
long_text_count = len(long_text)

In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherzyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
docs = long_text

## Vectorize by words in docs

In [131]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [132]:
words = []
for word in docs[0].split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(docs[0]))

['Louis', 'J.', 'Russell', '(circa', '1912–1973)', 'was', 'an', 'agent', 'of', 'the', 'Federal', 'Bureau', 'of', 'Investigation', '(FBI),', 'senior', 'investigator', 'of', 'the', 'House', 'Un-American', 'Activities', 'Committee', '(HUAC),', 'and', 'private', 'detective.\n==Background==\nLouis', 'James', 'Russell', 'graduated', 'from', 'the', 'Catholic', 'University', 'of', 'America.\n==Career==\nIn', '1935,', 'Russell', 'joined', 'the', 'FBI', 'as', 'an', 'agent.\n===HUAC===\nIn', '1945,', 'Russell', 'joined', 'HUAC', 'as', 'an', 'investigator.', '', 'Robert', 'E.', 'Stripling', 'has', 'Russell', 'testify', 'on', 'what', 'he', 'knew', 'about', 'Gerhart', 'Eisler', 'and', 'Hollywood', 'industry', 'people.', '', 'He', 'also', 'testified', 'about', 'Leon', 'Josephson', 'and', 'Alexander', 'Koral.', '', 'By', '1948,', 'Russell', 'was', 'a', 'HUAC', 'senior', 'investigator', 'in', 'the', 'Hiss-Chambers', 'case.', '', 'In', 'his', 'memoir', 'Six', 'Crises,', 'Richard', 'Nixon', 'recalled', '

In [157]:
processed_docs = list(map(lambda x: preprocess(x), docs))

In [158]:
len(processed_docs)

10102

In [159]:
model = gensim.models.Word2Vec(processed_docs, sg=1, size=300)

2019-06-26 16:51:28,610 : INFO : collecting all words and their counts
2019-06-26 16:51:28,612 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-26 16:51:28,864 : INFO : PROGRESS: at sentence #10000, processed 1214001 words, keeping 95752 word types
2019-06-26 16:51:28,868 : INFO : collected 96426 word types from a corpus of 1225961 raw words and 10102 sentences
2019-06-26 16:51:28,868 : INFO : Loading a fresh vocabulary
2019-06-26 16:51:28,952 : INFO : effective_min_count=5 retains 20958 unique words (21% of original 96426, drops 75468)
2019-06-26 16:51:28,953 : INFO : effective_min_count=5 leaves 1110763 word corpus (90% of original 1225961, drops 115198)
2019-06-26 16:51:29,007 : INFO : deleting the raw counts dictionary of 96426 items
2019-06-26 16:51:29,009 : INFO : sample=0.001 downsamples 21 most-common words
2019-06-26 16:51:29,010 : INFO : downsampling leaves estimated 1091580 word corpus (98.3% of prior 1110763)
2019-06-26 16:51:29,067 : INFO 

In [160]:
model.train(processed_docs, total_examples=model.corpus_count, epochs=model.epochs)

2019-06-26 16:52:11,825 : WARNING : Effective 'alpha' higher than previous training cycles
2019-06-26 16:52:11,826 : INFO : training model with 3 workers on 20958 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2019-06-26 16:52:12,832 : INFO : EPOCH 1 - PROGRESS: at 20.44% examples, 231561 words/s, in_qsize 5, out_qsize 0
2019-06-26 16:52:13,844 : INFO : EPOCH 1 - PROGRESS: at 43.35% examples, 235834 words/s, in_qsize 5, out_qsize 0
2019-06-26 16:52:14,869 : INFO : EPOCH 1 - PROGRESS: at 65.64% examples, 242884 words/s, in_qsize 5, out_qsize 0
2019-06-26 16:52:15,896 : INFO : EPOCH 1 - PROGRESS: at 90.77% examples, 245388 words/s, in_qsize 5, out_qsize 0
2019-06-26 16:52:16,204 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-26 16:52:16,226 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-26 16:52:16,232 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-26 16:52:16,232 : INFO : E

(5458285, 6129805)

In [102]:
#model.wv['paris']

In [54]:
#model.wv.similarity('paris','france')

0.5680741

## Vectorize documents 

In [135]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [136]:
type(processed_docs)

list

In [137]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_docs)
dictionary.save('data/wiki_set1.dict')  # store the dictionary, for future reference
print(dictionary)

2019-06-26 16:25:15,611 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-26 16:25:16,563 : INFO : adding document #10000 to Dictionary(95752 unique tokens: ['accord', 'activities', 'actor', 'agency', 'agenda']...)
2019-06-26 16:25:16,574 : INFO : built Dictionary(96426 unique tokens: ['accord', 'activities', 'actor', 'agency', 'agenda']...) from 10102 documents (total 1225961 corpus positions)
2019-06-26 16:25:16,583 : INFO : saving Dictionary object under data/wiki_set1.dict, separately None
/Users/sherzyang/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-06-26 16:25:16,633 : INFO : saved data/wiki_set1.dict


Dictionary(96426 unique tokens: ['accord', 'activities', 'actor', 'agency', 'agenda']...)


In [138]:
dictionary.token2id

{'accord': 0,
 'activities': 1,
 'actor': 2,
 'agency': 3,
 'agenda': 4,
 'agent': 5,
 'agents': 6,
 'aide': 7,
 'alch': 8,
 'alcoholic': 9,
 'alexander': 10,
 'alger': 11,
 'alumni': 12,
 'america': 13,
 'american': 14,
 'anderson': 15,
 'archive': 16,
 'assassinations': 17,
 'associate': 18,
 'atomic': 19,
 'attack': 20,
 'attorney': 21,
 'author': 22,
 'background': 23,
 'bedell': 24,
 'bernard': 25,
 'book': 26,
 'borrow': 27,
 'break': 28,
 'bring': 29,
 'bureau': 30,
 'bush': 31,
 'call': 32,
 'campaign': 33,
 'career': 34,
 'case': 35,
 'cash': 36,
 'catholic': 37,
 'chair': 38,
 'chairman': 39,
 'chamber': 40,
 'characterize': 41,
 'check': 42,
 'circa': 43,
 'circumstances': 44,
 'clients': 45,
 'committee': 46,
 'communist': 47,
 'company': 48,
 'conference': 49,
 'congressional': 50,
 'contract': 51,
 'credibility': 52,
 'crises': 53,
 'ctia': 54,
 'death': 55,
 'deaths': 56,
 'decade': 57,
 'deep': 58,
 'detective': 59,
 'die': 60,
 'director': 61,
 'discuss': 62,
 'dismiss

In [145]:
corpus = [dictionary.doc2bow(text) for text in processed_docs]

In [146]:
corpora.MmCorpus.serialize('data/wiki_set1.mm', corpus) 

2019-06-26 16:30:45,110 : INFO : storing corpus in Matrix Market format to data/wiki_set1.mm
2019-06-26 16:30:45,117 : INFO : saving sparse matrix to data/wiki_set1.mm
2019-06-26 16:30:45,118 : INFO : PROGRESS: saving document #0
2019-06-26 16:30:45,238 : INFO : PROGRESS: saving document #1000
2019-06-26 16:30:45,345 : INFO : PROGRESS: saving document #2000
2019-06-26 16:30:45,442 : INFO : PROGRESS: saving document #3000
2019-06-26 16:30:45,529 : INFO : PROGRESS: saving document #4000
2019-06-26 16:30:45,623 : INFO : PROGRESS: saving document #5000
2019-06-26 16:30:45,723 : INFO : PROGRESS: saving document #6000
2019-06-26 16:30:45,809 : INFO : PROGRESS: saving document #7000
2019-06-26 16:30:45,895 : INFO : PROGRESS: saving document #8000
2019-06-26 16:30:45,985 : INFO : PROGRESS: saving document #9000
2019-06-26 16:30:46,082 : INFO : PROGRESS: saving document #10000
2019-06-26 16:30:46,092 : INFO : saved 10102x96426 matrix, density=0.075% (727212/974095452)
2019-06-26 16:30:46,094 : 

In [140]:
# pip install spacy

In [141]:
# python -m spacy download en

In [147]:
import spacy
nlp = spacy.load('en')

In [166]:
#doc1 = nlp(processed_docs[0])
#doc2 = nlp(processed_docs[1])
doc1 = nlp(long_text[0])
doc2 = nlp(long_text[12])

In [167]:
print (doc1.similarity(doc2))


0.9569170543269897


/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


In [181]:
import operator

def compare_docs(text):
    all_scores =[]
    nlp = spacy.load('en')
    for i in range(len(long_text[0:10])):
        doc1 = nlp(text)
        doc2 = nlp(long_text[i])
        score = doc1.similarity(doc2)
        all_scores.append((i,score))
    
    scores_dict = dict(all_scores)
    
    return max(scores_dict.items(), key=operator.itemgetter(1))

In [182]:
compare_docs(long_text[19])

/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors

(4, 0.9421908017484784)

## Complexity

In [10]:
import textstat
import pandas as pd

In [11]:
complexity = [textstat.flesch_kincaid_grade(doc) for doc in docs]

In [12]:
df_complexity = pd.DataFrame()

In [13]:
df_complexity['score'] = pd.Series(complexity)

In [14]:
df_complexity['text'] = pd.Series(docs)

In [15]:
df_complexity.head(2)

,score,text
0,14.6,Louis J. Russell (circa 1912–1973) was an agen...
1,11.1,"Mary Holloway Wilhite (February 3, 1831 – Febr..."


In [64]:
df_complexity['score'].describe()

count    10102.000000
mean        15.854791
std         16.623426
min          1.900000
25%         10.800000
50%         13.200000
75%         17.100000
max       1046.200000
Name: score, dtype: float64

In [65]:
#Sorting  
four = []
three = []
two = []
one = []
for row in df_complexity['score']:
    if row > 15: 
        four.append(row)
    elif 15 >= row > 10:
        three.append(row)
    elif 10 < row <= 5:
        two.append(row)
    elif row <= 5:
        one.append(row)
        

In [66]:
#Sorting 
print("len_one:" + str(len(one)) + " , len_two:" + str(len(two)) 
      + " , len_three:" + str(len(three)) + " , len_four:" + str(len(four)))

len_one:43 , len_two:0 , len_three:4601 , len_four:3672


In [70]:
num_words = [textstat.lexicon_count(doc, removepunct=True) for doc in docs] 

In [71]:
df_complexity['num_words'] = pd.Series(num_words)

In [72]:
num_sent = [textstat.sentence_count(doc) for doc in docs]

In [73]:
df_complexity['num_sent'] = pd.Series(num_sent)

In [ ]:
#fog = [textstat.gunning_fog(doc) for doc in docs]

In [ ]:
#df_complexity['fog'] = pd.Series(fog)

In [ ]:
#smog = [textstat.smog_index(doc) for doc in docs]

In [ ]:
#df_complexity['smog'] = pd.Series(smog)

In [74]:
read_level = [textstat.automated_readability_index(doc) for doc in docs]

In [75]:
df_complexity['read_level'] = pd.Series(read_level)

In [ ]:
#c_liau = [textstat.coleman_liau_index(doc) for doc in docs]

In [ ]:
#df_complexity['c_liau'] = pd.Series(c_liau)

In [ ]:
#linsear = [textstat.linsear_write_formula(doc) for doc in docs]

In [ ]:
#df_complexity['linsear'] = pd.Series(linsear)

In [ ]:
#d_chall = [textstat.dale_chall_readability_score(doc) for doc in docs]

In [ ]:
#df_complexity['d_chall'] = pd.Series(d_chall)

In [76]:
### This is takes into account all of the tests 
standard_score = [textstat.text_standard(doc, float_output=False) for doc in docs]

In [77]:
df_complexity['standard_score'] = pd.Series(standard_score)

In [38]:
df_complexity.head(5)

,score,text
0,14.6,Louis J. Russell (circa 1912–1973) was an agen...
1,11.1,"Mary Holloway Wilhite (February 3, 1831 – Febr..."
2,11.6,Central African Republic–Democratic Republic o...
3,6.4,"Herma Svozilová-Johnová (August 29, 1914 &ndas..."
4,18.9,Sandy Frances Duncan is a Canadian writer of n...


'Temnoye (links=no|Темное) is a rural locality (a village) in Chernushinsky District, Perm Krai, Russia. The population was 83 as of 2010.\n== References ==\nRural localities in Perm Krai'

## First Model 

In [16]:
import spacy
import textstat
import operator

def give_harder_level(text):
    """
    Takes in text and returns an easier level read
    """
    ##find all the articles that are easier than this article 
    all_easier_index = []
    input_score = textstat.flesch_kincaid_grade(text)
    for i in range(len(df_complexity['score'])):
        if df_complexity['score'][i] > input_score:
            all_easier_index.append(i)
    
    #return all_easier_index
    
    ##find the article most similar to this one
    all_scores =[]
    nlp = spacy.load('en')
    for i in all_easier_index[0:30]:
        doc1 = nlp(text)
        doc2 = nlp(long_text[i])
        score = doc1.similarity(doc2)
        all_scores.append((i,score))
    
    scores_dict = dict(all_scores)
    
    best = max(scores_dict.items(), key=operator.itemgetter(1))
    print(best)
    index_best = list(best)[0]
    return df_complexity['text'][index_best]

In [17]:
example_text = "A watermelon is a type of edible fruit, they are 92% water. About 6% of a watermelon is sugar, which makes it very sweet. There are many different types of watermelon. Some have a green rind on the outside and a red-pink flesh on the inside, with brown seeds. Some can have yellow flesh, and some can be seedless. The green rind on the outside is not usually eaten, though it can be used as a vegetable. It can also be stewed or pickled. Most watermelons are oblong or spherical. In Japan, watermelons are grown in different shapes. Many people like to eat watermelon in the summer because the fruit is cool and refreshing. Watermelons are a great source of vitamin A, vitamin C, vitamin B6 and vitamin B1. They also contain potassium, magnesium, carotenoid antioxidant, and lycopene. Watermelons are fruits that come from a vine-like plant."

In [18]:
give_harder_level(example_text)

/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors

/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors

(6, 0.8740465692490009)


/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/Users/sherzyang/anaconda3/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors

"Hamiltonian simulation (also referred to as quantum simulation) is a problem in quantum information science that attempts to find the computational complexity and quantum algorithms needed for simulating quantum systems. Hamiltonian simulation is a problem that demands algorithms which implement the evolution of a quantum state efficiently. The Hamiltonian simulation problem was proposed by Richard Feynman in 1982, where he proposed a quantum computer as a possible solution since the simulation of general Hamiltonians seem to grow exponentially with respect to the system size. \n== Problem statement ==\nIn the Hamiltonian simulation problem, given a Hamiltonian H (2^n \\times 2^n [matrix] acting on  qubits, a time t and maximum simulation error \\epsilon, the goal is to find an algorithm that approximates  U  such that ||U - e^{-iht} || \\leq \\epsilon . where e^{-iht} is the ideal evolution.\nA special case of the Hamiltonian simulation problem is the local Hamiltonian simulation pro

In [56]:
docs[19]

"The 2019 Conference Carolinas Men's Volleyball Tournament was the men's volleyball tournament for Conference Carolinas during the 2019 NCAA Division I & II men's volleyball season. It was held April 13 through April 18, 2019 at campus sites. The winner received the conference's automatic bid to the 2019 NCAA Volleyball Tournament.\n==Seeds==\nEight of the nine teams are eligible for the postseason, with the highest seed hosting each round. Teams are seeded by record within the conference, with a tiebreaker system to seed teams with identical conference records.\n==Schedule and results==\n==Bracket==\nThe win clinched Barton's second trip to the NCAA Tournament.\n==References==\n2019 Conference Carolinas men's volleyball season\n2019 NCAA Division I & II men's volleyball season\nVolleyball competitions in the United States"

In [57]:
import wikipedia

In [60]:
ny = wikipedia.page("New York City")
new_york_city = ny.content

In [ ]:
### I should be able to test the effectiveness 